In [21]:
import pandas as pd
import fastparquet


In [22]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(2940, 16)

In [23]:
len(df[df.tiling_compatible == True]) 

75

In [24]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format   1203
1       failed_to_extract    647
2          cant_open_file    551
3         group_structure    191
4  tile_generation_failed    179
5        no_xy_dimensions     79
6                    None     75
7  cant_extract_variables     15


In [39]:
df[df.incompatible_reason == 'tile_generation_failed'].error_message.unique()


array(['list index out of range',
       'Expecting value: line 1 column 1 (char 0)',
       'titiler.xarray can only work with 2D or 3D dataset',
       "No group found in store <fsspec.mapping.FSMap object at 0x7fef56597ef0> at path ''",
       "unable to decode time units 'months since 1999-01-01 00:00:00' with 'the default calendar'. Try opening your dataset with decode_times=False or installing cftime if it is not installed.",
       "unable to decode time units 'months since 2010-01-01' with 'the default calendar'. Try opening your dataset with decode_times=False or installing cftime if it is not installed.",
       "unable to decode time units 'months since 2011-01-01' with 'the default calendar'. Try opening your dataset with decode_times=False or installing cftime if it is not installed.",
       "b'CDF\\x01\\x00\\x00\\x00\\x00' is not the signature of a valid netCDF4 file",
       'Unable to allocate 15.7 GiB for an array with shape (12, 13121, 26811) and data type float32',


In [ ]:
list(df[df.error_message == "unable to decode time units 'months since 2010-01-01' with 'the default calendar'. Try opening your dataset with decode_times=False or installing cftime if it is not installed."].collection_concept_id)

['C1368522053-GES_DISC']

In [33]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

27                                               [SWIR1]
29                                               [TIRS2]
34     [latitude, longitude, scan_datetime, year, mon...
37     [aerosol_optical_thickness_550_land, aerosol_o...
43     [Aerosol_Optical_Thickness_550_Expected_Uncert...
                             ...                        
623    [Delta_Time, SZA, VZA, SAz, VAz, Longitude, La...
625                                             [weight]
640    [obs_id, fov_obs_id, obs_time_tai93, obs_time_...
643    [EFLUXICE, EFLUXWTR, FRSEAICE, HFLUXICE, HFLUX...
646                    [OriginalInputFiles, XmlMetadata]
Name: data_variables, Length: 160, dtype: object

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
